Comparison of Fasttext and Word2vec

In [1]:
# imports
import pandas as pd
import numpy as np
import os
import sklearn

In [2]:
# downloads
import nltk
nltk.download('brown')

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [3]:
local_data_root = "C:/Users/lenovo/Documents/"
data_path = local_data_root + 'data/'
code_path = local_data_root + 'NLP/'

In [5]:
not os.path.isfile(data_path+'brown_corp.txt')

False

In [6]:
# generate brown corpus text file
import os
if not os.path.isfile(data_path+'brown_corp.txt'):
    with open(data_path+'brown_corp.txt', 'w+') as f:
        for word in nltk.corpus.brown.words():
            f.write('{word}'.format(word=word))
    f.close()

In [9]:
import os
os.listdir(data_path)
os.chdir(data_path)
!ls -la

total 4856
drwxr-xr-x 1 lenovo 197121       0 May 28 12:06 .
drwxr-xr-x 1 lenovo 197121       0 May 28 12:06 ..
-rw-r--r-- 1 lenovo 197121 4965882 May 28 12:05 brown_corp.txt


In [ ]:
if (not os.path.isfile(data_path+'text8.zip')) & (not os.path.isfile(data_path+'text8.txt')):
    !curl http://mattmahoney.net/dc/text8.zip
    !unzip text8.zip

In [ ]:
# cannot run this
if (not os.path.isfile(data_path+'text9.zip')) & (not os.path.isfile(data_path+'text9.txt')):
    !curl http://mattmahoney.net/dc/enwik9.zip
    !unzip enwik9.zip
    !perl {FT_HOME}wikifil.pl enwik9 > text9

In [16]:
MODELS_DIR = data_path+'models/'
if not os.path.isdir(MODELS_DIR):
    os.chdir(data_path)
    !mkdir models

In [17]:
print(os.path.isdir(MODELS_DIR))
print(MODELS_DIR)

True
C:/Users/lenovo/Documents/data/models/


In [25]:
lr = 0.05
dim = 100
ws = 5
epoch = 5
minCount = 1
neg = 5
loss = 'ns'
t = 1e-4

from gensim.models import Word2Vec
from gensim.models.word2vec import Text8Corpus

# Same values as used for fastText training above
params = {
    'alpha': lr,
    'size': dim,
    'window': ws,
    'iter': epoch,
    'min_count': minCount,
    'sample': t,
    'sg': 1,
    'hs': 0,
    'negative': neg
}

def train_models(corpus_file, output_name):
        
    output_file = '{:s}_gs'.format(output_name)
    if not os.path.isfile(os.path.join(MODELS_DIR, '{:s}.vec'.format(output_file))):
        print('\nTraining word2vec on {:s} corpus..'.format(corpus_file))
        
        # Text8Corpus class for reading space-separated words file
        %time gs_model = Word2Vec(Text8Corpus(corpus_file), **params); gs_model
        # Direct local variable lookup doesn't work properly with magic statements (%time)
        locals()['gs_model'].wv.save_word2vec_format(os.path.join(MODELS_DIR, '{:s}.vec'.format(output_file)))
        print('\nSaved gensim model as {:s}.vec'.format(output_file))
    else:
        print('\nUsing existing model file {:s}.vec'.format(output_file))

evaluation_data = {}
train_models(data_path+'brown_corp.txt', 'brown')


Training word2vec on C:/Users/lenovo/Documents/data/brown_corp.txt corpus..
Wall time: 33.4 s

Saved gensim model as brown_gs.vec


In [26]:
WV_DIR = local_data_root + 'word2vec/'

In [27]:
print(WV_DIR)
print(os.path.isdir(WV_DIR))

C:/Users/lenovo/Documents/word2vec/
True


In [38]:
import gensim.models as gsm
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Training times in seconds
evaluation_data['brown'] = [(33.4)]

def print_accuracy(model, questions_file):
    print('Evaluating...\n')
    acc = model.accuracy(questions_file)

    sem_correct = sum((len(acc[i]['correct']) for i in range(5)))
    sem_total = sum((len(acc[i]['correct']) + len(acc[i]['incorrect'])) for i in range(5))
    sem_acc = 100*float(sem_correct)/sem_total
    print('\nSemantic: {:d}/{:d}, Accuracy: {:.2f}%'.format(sem_correct, sem_total, sem_acc))
    
    syn_correct = sum((len(acc[i]['correct']) for i in range(5, len(acc)-1)))
    syn_total = sum((len(acc[i]['correct']) + len(acc[i]['incorrect'])) for i in range(5,len(acc)-1))
    syn_acc = 100*float(syn_correct)/syn_total
    print('Syntactic: {:d}/{:d}, Accuracy: {:.2f}%\n'.format(syn_correct, syn_total, syn_acc))
    return (sem_acc, syn_acc)

word_analogies_file = WV_DIR+'questions-words.txt'
accuracies = []
print('\nLoading Gensim embeddings')
brown_gs = gsm.KeyedVectors.load_word2vec_format(MODELS_DIR + 'brown_gs.vec')
print('Accuracy for Word2Vec:')
accuracies.append(print_accuracy(brown_gs, word_analogies_file))

2018-05-28 13:34:16,066 : INFO : loading projection weights from C:/Users/lenovo/Documents/data/models/brown_gs.vec
2018-05-28 13:34:16,159 : INFO : loaded (1, 100) matrix from C:/Users/lenovo/Documents/data/models/brown_gs.vec



Loading Gensim embeddings
Accuracy for Word2Vec:
Evaluating...



ZeroDivisionError: float division by zero

In [31]:
import gensim.models as gsm
model = gsm.KeyedVectors.load_word2vec_format(MODELS_DIR + 'brown_gs.vec')
questions_file = WV_DIR+'questions-words.txt'

In [32]:
help(model.accuracy)

Help on method accuracy in module gensim.models.keyedvectors:

accuracy(questions, restrict_vocab=30000, most_similar=<function WordEmbeddingsKeyedVectors.most_similar at 0x0000000BAADD7400>, case_insensitive=True) method of gensim.models.keyedvectors.Word2VecKeyedVectors instance
    Compute accuracy of the model. `questions` is a filename where lines are
    4-tuples of words, split into sections by ": SECTION NAME" lines.
    See questions-words.txt in
    https://storage.googleapis.com/google-code-archive-source/v2/code.google.com/word2vec/source-archive.zip
    for an example.
    
    The accuracy is reported (=printed to log and returned as a list) for each
    section separately, plus there's one aggregate summary at the end.
    
    Use `restrict_vocab` to ignore all questions containing a word not in the first `restrict_vocab`
    words (default 30,000). This may be meaningful if you've sorted the vocabulary by descending frequency.
    In case `case_insensitive` is True, th

In [33]:
acc = model.accuracy(questions_file)

In [34]:
print(type(acc))

<class 'list'>


In [35]:
print(len(acc))

15


In [37]:
acc

[{'correct': [], 'incorrect': [], 'section': 'capital-common-countries'},
 {'correct': [], 'incorrect': [], 'section': 'capital-world'},
 {'correct': [], 'incorrect': [], 'section': 'currency'},
 {'correct': [], 'incorrect': [], 'section': 'city-in-state'},
 {'correct': [], 'incorrect': [], 'section': 'family'},
 {'correct': [], 'incorrect': [], 'section': 'gram1-adjective-to-adverb'},
 {'correct': [], 'incorrect': [], 'section': 'gram2-opposite'},
 {'correct': [], 'incorrect': [], 'section': 'gram3-comparative'},
 {'correct': [], 'incorrect': [], 'section': 'gram4-superlative'},
 {'correct': [], 'incorrect': [], 'section': 'gram5-present-participle'},
 {'correct': [], 'incorrect': [], 'section': 'gram6-nationality-adjective'},
 {'correct': [], 'incorrect': [], 'section': 'gram7-past-tense'},
 {'correct': [], 'incorrect': [], 'section': 'gram8-plural'},
 {'correct': [], 'incorrect': [], 'section': 'gram9-plural-verbs'},
 {'correct': [], 'incorrect': [], 'section': 'total'}]